# 路由查询引擎

多选效果不明显，而且可能会很慢。

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 3.41 s, sys: 404 ms, total: 3.82 s
Wall time: 3.46 s


In [18]:
import nest_asyncio

nest_asyncio.apply()

## 加载数据

In [2]:
%%time

items=[
    "颐和园",
    "北海公园",
    "故宫",
]

from llama_index.readers.web import TrafilaturaWebReader

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 751 ms, sys: 3.79 ms, total: 755 ms
Wall time: 994 ms


3

In [3]:
%%time

nodes = Settings.node_parser.get_nodes_from_documents(documents)

CPU times: user 83.8 ms, sys: 0 ns, total: 83.8 ms
Wall time: 83 ms


## 在相同数据上建立摘要索引和向量索引

In [4]:
%%time


from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

CPU times: user 2.5 s, sys: 119 ms, total: 2.61 s
Wall time: 54.7 s


## 定义查询引擎和元数据

In [5]:
%%time


list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

CPU times: user 30.8 ms, sys: 3.95 ms, total: 34.8 ms
Wall time: 34.2 ms


In [6]:
%%time

from llama_index.core.tools import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "对景点问题的总结问题很有用。"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于景点相关的具体问题。"
    ),
)

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 16.7 µs


### 单选

In [7]:
%%time

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

CPU times: user 190 µs, sys: 0 ns, total: 190 µs
Wall time: 192 µs


In [8]:
%%time

response = query_engine.query("介绍下颐和园")
print(str(response))

颐和园是中国清朝时期的皇家园林，位于北京市西郊海淀区新建宫门路19号。它最初名为清漪园，始建于乾隆十五年（1750年），但在咸丰十年（1860年）被英法联军烧毁。

颐和园以其利用昆明湖、万寿山为基址的独特布局而闻名，其设计灵感来源于杭州西湖的风景，并融合了江南园林的设计手法和意境。它被誉为“皇家园林博物馆”，是保存最完好的一座皇家行宫御苑。整个园区占地3.0平方公里，展现了中国古典园林艺术的魅力。
CPU times: user 86.9 ms, sys: 0 ns, total: 86.9 ms
Wall time: 5.36 s


In [9]:
%%time

response = query_engine.query("颐和园的门票价格？")
response.response

CPU times: user 34.3 ms, sys: 0 ns, total: 34.3 ms
Wall time: 1.48 s


'颐和园的门票价格在旺季为30元/张，在淡季为20元/张。'

In [10]:
%%time

response = query_engine.query("介绍下颐和园，以及门票价格")
response.response

CPU times: user 33.9 ms, sys: 0 ns, total: 33.9 ms
Wall time: 5.24 s


'颐和园是中国著名的皇家园林之一，位于北京市。它拥有丰富的历史背景和美丽的自然景观，是游客游览北京的热门景点。\n\n在不同的季节，颐和园有不同的门票价格：\n- 在旺季（通常指春季、夏季和秋季），单人门票的价格为30元人民币。\n- 在淡季（一般指冬季），单人门票的价格为20元人民币。\n\n此外，颐和园还提供联票，包含入园费用以及访问部分特色景点的权限。联票价格如下：\n- 在旺季，联票的价格为60元人民币。\n- 在淡季，联票的价格为50元人民币。\n\n这些特色景点包括德和园、颐和园博物馆、佛香阁和苏州街。其中，单独参观德和园需要支付5元人民币，访问颐和园博物馆需20元人民币，佛香阁的门票是10元人民币，而苏州街则为10元人民币。\n\n通过购买联票，游客可以更方便地游览这些景点，享受更多优惠。'

### 多选

In [16]:
%%time

from llama_index.core.selectors import LLMMultiSelector

query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)

CPU times: user 222 µs, sys: 17 µs, total: 239 µs
Wall time: 244 µs


In [19]:
%%time

response = query_engine.query("介绍下颐和园")
response.response

CPU times: user 490 ms, sys: 10.7 ms, total: 500 ms
Wall time: 2min 6s


'颐和园位于中国北京市西郊，是清朝时期的大型皇家园林。它以昆明湖与万寿山为基础，融合了中国古代园林艺术的精髓，并在乾隆年间进行了大规模扩建与改造，最终形成了一座集自然景观与人工建筑于一体的宏伟园区。\n\n### 历史沿革\n颐和园最初名为“清漪园”，始建于1750年左右，作为皇帝休闲娱乐的场所。然而，在第二次鸦片战争中，它遭到英法联军破坏。战后，清朝政府决定重建，并改名“颐和园”。在光绪时期进行了大规模扩建，并增加了许多新的建筑。\n\n### 主要景点\n- **苏州街**：原称万寿买卖街，位于昆明湖西侧，是乾隆皇帝命人仿照江南水乡建造的商业街。\n  \n- **长廊**：全长约728米，是中国最长的画廊之一。沿湖而建，两侧绘有多幅山水人物、花鸟鱼虫等题材的彩画。\n\n- **佛香阁**：位于万寿山顶部，高约41米，八角形三层楼阁，内部供奉着佛像，是颐和园的标志性建筑。\n\n### 文化价值\n颐和园不仅展现了中国古典园林艺术的精湛技艺，还融合了佛教、道教以及民间艺术元素。它是中国传统文化与自然景观完美结合的典范，体现了五个多世纪以来中国最高权力中心的文化象征。作为世界文化遗产之一，颐和园不仅是历史的见证，也是现代人了解中国古代文化和建筑艺术的重要窗口。\n\n颐和园占地3.0平方公里，不仅拥有宏伟的建筑群，还保留了大量的历史遗迹，是中国传统文化的重要象征。'

In [20]:
%%time

response = query_engine.query("颐和园的门票价格？")
response.response

CPU times: user 34.6 ms, sys: 0 ns, total: 34.6 ms
Wall time: 1.65 s


'颐和园的门票价格在旺季为30元/张，在淡季为20元/张。'

In [21]:
%%time

response = query_engine.query("介绍下颐和园，以及门票价格")
response.response

CPU times: user 30.8 ms, sys: 4.27 ms, total: 35.1 ms
Wall time: 4.66 s


'颐和园是中国著名的皇家园林之一，位于北京。它分为两部分：主要区域和"园中园"。主要区域的门票在旺季时为30元/张，在淡季时为20元/张。如果购买联票，则价格在旺季为60元/张，在淡季为50元/张，联票包含了门票以及园中园的访问权限。\n\n"园中园"包括德和园、颐和园博物馆、佛香阁和苏州街。其中，德和园的门票是5元/张；颐和园博物馆的票价为20元/张；佛香阁的票价为10元/张；苏州街的票价也是10元/张。\n\n联票包含了上述所有景点的访问权限。'